In [1]:
import streamlit as st
import pandas as pd
import plotly.graph_objs as go
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler(feature_range=(0,1))

2023-07-08 13:37:03.213891: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-08 13:37:03.466838: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-08 13:37:05.417345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df_nse = pd.read_csv("/home/sherherd/Desktop/Assign/Dennoh/project.ipynb/NSE-Tata-Global-Beverages-Limited.csv")

df_nse["Date"] = pd.to_datetime(df_nse.Date, format="%Y-%m-%d")
df_nse.index = df_nse['Date']

data = df_nse.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0, len(df_nse)), columns=['Date', 'Close'])

In [3]:
for i in range(0, len(data)):
    new_data["Date"][i] = data['Date'][i]
    new_data["Close"][i] = data["Close"][i]

new_data.index = new_data.Date
new_data.drop("Date", axis=1, inplace=True)


In [4]:
dataset = new_data.values

train = dataset[0:987, :]
valid = dataset[987:, :]

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)


In [5]:
x_train, y_train = [], []

for i in range(60, len(train)):
    x_train.append(scaled_data[i-60:i, 0])
    y_train.append(scaled_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = load_model("/home/sherherd/Desktop/Assign/Dennoh/project.ipynb/saved_lstm_model.h5")

2023-07-08 13:37:12.937916: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-08 13:37:12.941474: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-08 13:37:12.946396: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [6]:
inputs = new_data[len(new_data)-len(valid)-60:].values
inputs = inputs.reshape(-1, 1)
inputs = scaler.transform(inputs)

X_test = []
for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)

In [7]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

train = new_data[:987]
valid = new_data[987:]
valid['Predictions'] = closing_price

2023-07-08 13:37:14.667392: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-08 13:37:14.671565: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-08 13:37:14.676826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

8/8 [==============================] - 3s 43ms/step


/tmp/ipykernel_5258/2862448319.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = closing_price


In [8]:
df = pd.read_csv("stock_data.csv")

In [13]:
st.title("Stock Price Analysis Dashboard")

tabs = ["NSE-TATAGLOBAL Stock Data", "Facebook Stock Data"]
selected_tab = st.selectbox("Select dataset", tabs)

if selected_tab == "NSE-TATAGLOBAL Stock Data":
    st.header("Actual closing price")
    fig_actual = go.Figure(data=go.Scatter(x=train.index, y=valid["Close"], mode='markers'))
    fig_actual.update_layout(title='Scatter plot', xaxis={'title': 'Date'}, yaxis={'title': 'Closing Rate'})
    st.plotly_chart(fig_actual)

    st.header("LSTM Predicted closing price")
    fig_predicted = go.Figure(data=go.Scatter(x=valid.index, y=valid["Predictions"], mode='markers'))
    fig_predicted.update_layout(title='Scatter plot', xaxis={'title': 'Date'}, yaxis={'title': 'Closing Rate'})
    st.plotly_chart(fig_predicted)

elif selected_tab == "Facebook Stock Data":
    st.header("Facebook Stocks High vs Lows")
    selected_stocks = st.multiselect("Select stocks", ['TSLA', 'AAPL', 'FB', 'MSFT'], default=['FB'])
    dropdown = {"TSLA": "Tesla", "AAPL": "Apple", "FB": "Facebook", "MSFT": "Microsoft"}

    fig_highlow = go.Figure()
    for stock in selected_stocks:
        fig_highlow.add_trace(go.Scatter(x=df[df["Stock"] == stock]["Date"],
                                         y=df[df["Stock"] == stock]["High"],
                                         mode='lines',
                                         opacity=0.7,
                                         name=f'High {dropdown[stock]}',
                                         textposition='bottom center'))
    fig_highlow.update_layout(title=f"High and Low Prices for {', '.join(str(dropdown[i]) for i in selected_stocks)} Over Time",
                              xaxis={"title": "Date",
                                     'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 'step': 'month',
                                                                          'stepmode': 'backward'},
                                                                         {'count': 6, 'label': '6M', 'step': 'month',
                                                                          'stepmode': 'backward'},
                                                                         {'step': 'all'}])},
                                     'rangeslider': {'visible': True},
                                     'type': 'date'},
                              yaxis={"title": "Price (USD)"})
    st.plotly_chart(fig_highlow)

    st.header("Facebook Market Volume")
    fig_volume = go.Figure()
    for stock in selected_stocks:
        fig_volume.add_trace(go.Scatter(x=df[df["Stock"] == stock]["Date"],
                                        y=df[df["Stock"] == stock]["Volume"],
                                        mode='lines',
                                        opacity=0.7,
                                        name=f'Volume {dropdown[stock]}',
                                        textposition='bottom center'))
    fig_volume.update_layout(title=f"Market Volume for {', '.join(str(dropdown[i]) for i in selected_stocks)} Over Time",
                             xaxis={"title": "Date",
                                    'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 'step': 'month',
                                                                         'stepmode': 'backward'},
                                                                        {'count': 6, 'label': '6M', 'step': 'month',
                                                                         'stepmode': 'backward'},
                                                                        {'step': 'all'}])},
                                    'rangeslider': {'visible': True},
                                    'type': 'date'},
                             yaxis={"title": "Transactions Volume"})
    st.plotly_chart(fig_volume)


2023-07-08 13:38:46.383 
  command:

    streamlit run /home/sherherd/.local/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
